### Lab: Create your collection of songs & audio features

To move forward witht the project, you need to create a collection of songs with their audio features - as large as possible! 

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster.

The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

## Spotipy API

Create an Spotify account and follow these steps to register an app: https://developer.spotify.com/documentation/general/guides/app-settings/

After the app is created, you can see it on your dashboard
https://developer.spotify.com/dashboard/applications

Click on it and you'll find the client id and client secret.

#### Authentification

In [2]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="e5e880fa9e7a48b9a128f347288238dd",
    client_secret="5fd1c32f02774dfe85e6b7c5c1a8c2e2"))

#### Pagination using "next"

When you collect songs from a playlist using `sp.playlist_tracks`, you're limited by the `limit` parameter, which has a maximum (and default) value of 100. When the playlist has more than 100 songs, you have to collect them by navigating through the "pages" of the results.

The parameter `offset` allows you to retrieve resuls starting at a certain position: if you start at position 101, you'd get the next "page" of results. An offset of 201 would give you the third page, and so on.

The function `sp.next()` does the same, but in a simpler way: it can be used on the results from any request to directly retrieve the results for the next page.

We can check whether there's a next page or not by accessing the key `next` on the results from any request.

In [4]:
tracks_from_playlist = sp.playlist_tracks("https://open.spotify.com/playlist/2DOhUKA4ADyLGtS41w9cnY")

In [8]:
tracks_from_playlist

{'href': 'https://api.spotify.com/v1/playlists/2DOhUKA4ADyLGtS41w9cnY/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-04-03T13:06:55Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1177318269'},
    'href': 'https://api.spotify.com/v1/users/1177318269',
    'id': '1177318269',
    'type': 'user',
    'uri': 'spotify:user:1177318269'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4hfcSstwnyuBoek1dQwLkG'},
       'href': 'https://api.spotify.com/v1/artists/4hfcSstwnyuBoek1dQwLkG',
       'id': '4hfcSstwnyuBoek1dQwLkG',
       'name': 'Cymande',
       'type': 'artist',
       'uri': 'spotify:artist:4hfcSstwnyuBoek1dQwLkG'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',

In [5]:
# this is the link to the "next" page
next_page_link = tracks_from_playlist["next"]
next_page_link

In [6]:
# this gives the full results directly
next_results = sp.next(tracks_from_playlist)
next_results

In [ ]:
# this playlist has 111 songs, so there's no 3rt page:
print(sp.next(next_results))
print(next_results["next"])

 ## Sample lab solution

#### Step 1: get uris of all the songs in a playlist (no matter how many songs the playlist has)

In [10]:
def get_tracks_ids_from_playlist(playlist_id):
    
    final_results = []
    result = sp.playlist_tracks(playlist_id)
    final_results.extend(result["items"])
    
    while result["next"]:
        
        result = sp.next(result)
        final_results.extend(result["items"])
        
    ids = [track["track"]["id"] for track in final_results]
    
    # we return only values that are not None:
    return [i for i in ids if i]

In [13]:
track_ids = get_tracks_ids_from_playlist("https://open.spotify.com/playlist/2DOhUKA4ADyLGtS41w9cnY")

In [15]:
len(track_ids)

59

#### Step 2: build a df with all the audio features

In [14]:
# we're gonna iterate through the output of the function above
sp.audio_features(track_ids[0])[0]

{'danceability': 0.789,
 'energy': 0.602,
 'key': 7,
 'loudness': -10.192,
 'mode': 0,
 'speechiness': 0.198,
 'acousticness': 0.3,
 'instrumentalness': 0.00549,
 'liveness': 0.0737,
 'valence': 0.804,
 'tempo': 109.923,
 'type': 'audio_features',
 'id': '6cBhKpoE1GNqEtwqGScyBy',
 'uri': 'spotify:track:6cBhKpoE1GNqEtwqGScyBy',
 'track_href': 'https://api.spotify.com/v1/tracks/6cBhKpoE1GNqEtwqGScyBy',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6cBhKpoE1GNqEtwqGScyBy',
 'duration_ms': 251400,
 'time_signature': 4}

In [20]:
import pandas as pd

In [21]:
def get_audio_features_df(playlist_id):
    track_ids = get_tracks_ids_from_playlist(playlist_id)
    aud_feat = [sp.audio_features(track)[0] for track in track_ids]
    return pd.DataFrame(aud_feat)

In [22]:
audio_f = get_audio_features_df("https://open.spotify.com/playlist/2DOhUKA4ADyLGtS41w9cnY")
audio_f.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.789,0.602,7,-10.192,0,0.1980,0.3000,0.005490,0.0737,0.804,109.923,audio_features,6cBhKpoE1GNqEtwqGScyBy,spotify:track:6cBhKpoE1GNqEtwqGScyBy,https://api.spotify.com/v1/tracks/6cBhKpoE1GNq...,https://api.spotify.com/v1/audio-analysis/6cBh...,251400,4
1,0.861,0.726,0,-6.991,1,0.0390,0.3860,0.000058,0.0417,0.981,121.551,audio_features,1X9povBxQkI1ojRA893fv8,spotify:track:1X9povBxQkI1ojRA893fv8,https://api.spotify.com/v1/tracks/1X9povBxQkI1...,https://api.spotify.com/v1/audio-analysis/1X9p...,175253,4
2,0.597,0.859,11,-6.343,0,0.0373,0.5060,0.000002,0.1870,0.908,172.968,audio_features,1b7tZPt3S2UN6OrPUBqGCp,spotify:track:1b7tZPt3S2UN6OrPUBqGCp,https://api.spotify.com/v1/tracks/1b7tZPt3S2UN...,https://api.spotify.com/v1/audio-analysis/1b7t...,192600,4
3,0.789,0.683,8,-8.835,1,0.0860,0.4800,0.000000,0.3500,0.882,92.311,audio_features,4pW3aTX3kAmOaPJfc9UM12,spotify:track:4pW3aTX3kAmOaPJfc9UM12,https://api.spotify.com/v1/tracks/4pW3aTX3kAmO...,https://api.spotify.com/v1/audio-analysis/4pW3...,233613,4
4,0.829,0.435,0,-10.136,1,0.0515,0.0363,0.878000,0.1190,0.594,103.816,audio_features,5VarZFY00sO54HErJ3TPCI,spotify:track:5VarZFY00sO54HErJ3TPCI,https://api.spotify.com/v1/tracks/5VarZFY00sO5...,https://api.spotify.com/v1/audio-analysis/5Var...,230147,4


#### Step 3: grow the df with multiple playlists

In [23]:
playlists = ["https://open.spotify.com/playlist/2sJmgI7nywfWjO6jEz8nSW",
             "https://open.spotify.com/playlist/0yqyjuBaPbIXDrw1F1dCfx",
             "https://open.spotify.com/playlist/2DOhUKA4ADyLGtS41w9cnY",
             "https://open.spotify.com/playlist/36556iTTUVxgYR94ST7kdW"]

In [24]:
def get_big_audio_df(playlists_list):
    audio_f_df = pd.DataFrame()
    
    for p in playlists_list:
        audio_f_df = audio_f_df.append(get_audio_features_df(p))

    return audio_f_df

In [25]:
big_df_songs_audiofeat = get_big_audio_df(playlists)

In [26]:
big_df_songs_audiofeat

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.339,0.1970,10,-13.937,0,0.0362,0.85200,0.015000,0.1110,0.0385,91.330,audio_features,65N9abRNej5yT3nuE2jEcE,spotify:track:65N9abRNej5yT3nuE2jEcE,https://api.spotify.com/v1/tracks/65N9abRNej5y...,https://api.spotify.com/v1/audio-analysis/65N9...,266733,1
1,0.509,0.2880,5,-11.317,0,0.0424,0.67100,0.000607,0.1570,0.3750,143.897,audio_features,3fVoaJNF8ALQlL0MG5XpWa,spotify:track:3fVoaJNF8ALQlL0MG5XpWa,https://api.spotify.com/v1/tracks/3fVoaJNF8ALQ...,https://api.spotify.com/v1/audio-analysis/3fVo...,275827,3
2,0.550,0.3730,9,-8.772,1,0.0314,0.39100,0.055100,0.0935,0.3300,121.941,audio_features,5AdpHyyqm66NhW0wpZg5yK,spotify:track:5AdpHyyqm66NhW0wpZg5yK,https://api.spotify.com/v1/tracks/5AdpHyyqm66N...,https://api.spotify.com/v1/audio-analysis/5Adp...,293213,3
3,0.228,0.0678,0,-19.599,1,0.0340,0.98600,0.261000,0.1120,0.1890,99.888,audio_features,1sUI0jcWbtUcpdNxITijPM,spotify:track:1sUI0jcWbtUcpdNxITijPM,https://api.spotify.com/v1/tracks/1sUI0jcWbtUc...,https://api.spotify.com/v1/audio-analysis/1sUI...,100267,3
4,0.542,0.5190,11,-14.059,0,0.5130,0.39700,0.011600,0.1060,0.3120,155.185,audio_features,3R1abI6qZDg9d3CI4oLDnf,spotify:track:3R1abI6qZDg9d3CI4oLDnf,https://api.spotify.com/v1/tracks/3R1abI6qZDg9...,https://api.spotify.com/v1/audio-analysis/3R1a...,366480,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,0.700,0.7800,11,-10.303,0,0.0726,0.06820,0.820000,0.0599,0.5920,120.007,audio_features,3oGoOyqFgmZ6rV2y3Zgozm,spotify:track:3oGoOyqFgmZ6rV2y3Zgozm,https://api.spotify.com/v1/tracks/3oGoOyqFgmZ6...,https://api.spotify.com/v1/audio-analysis/3oGo...,409292,4
39,0.797,0.8590,2,-10.725,1,0.0568,0.17300,0.901000,0.3790,0.2100,118.003,audio_features,5k0P8XWm6m7PCzCa7TEzOC,spotify:track:5k0P8XWm6m7PCzCa7TEzOC,https://api.spotify.com/v1/tracks/5k0P8XWm6m7P...,https://api.spotify.com/v1/audio-analysis/5k0P...,243680,4
40,0.796,0.8600,9,-5.917,0,0.0464,0.03630,0.727000,0.0681,0.5850,125.991,audio_features,7uEyAGmzxHFX4l5lsHpewa,spotify:track:7uEyAGmzxHFX4l5lsHpewa,https://api.spotify.com/v1/tracks/7uEyAGmzxHFX...,https://api.spotify.com/v1/audio-analysis/7uEy...,406798,4
41,0.773,0.7710,5,-7.063,1,0.0453,0.04960,0.659000,0.1110,0.9620,105.801,audio_features,6nPnAYsGwE8HDGxdNvSaex,spotify:track:6nPnAYsGwE8HDGxdNvSaex,https://api.spotify.com/v1/tracks/6nPnAYsGwE8H...,https://api.spotify.com/v1/audio-analysis/6nPn...,273059,4


Disclaimer: when Spotify's API takes more than 5 seconds to respond, it gives a Time readout error - ideally we'd use try - except to circumvent that. It can be in our next iteration!